In [ ]:
# Scraping 
# https://www.skiresort.info/ski-resorts/usa/

In [1]:
# Import dependencies
import pandas as pd
import requests
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
from webdriver_manager.chrome import ChromeDriverManager
from pprint import pprint

In [2]:
request = requests.get("https://www.skiresort.info/ski-resorts/usa/")

In [3]:
prep_soup = bs(request.text, 'html.parser')

In [4]:
panel = prep_soup.find('div', class_= "panel panel-primary")

In [5]:
# pprint(panel)

In [20]:
pages = prep_soup.find('ul', class_= "pagination pull-right")

In [21]:
print(pages)

<ul class="pagination pull-right" id="pagebrowser1"> <li> <a href="https://www.skiresort.info/ski-resorts/usa/">«</a> </li> <li> <a href="https://www.skiresort.info/ski-resorts/usa/">‹</a> </li> <li class="active"> <a href="https://www.skiresort.info/ski-resorts/usa/">1</a> </li> <li class=""> <a href="https://www.skiresort.info/ski-resorts/usa/page/2/">2</a> </li> <li class=""> <a href="https://www.skiresort.info/ski-resorts/usa/page/3/">3</a> </li> <li class=""> <a href="https://www.skiresort.info/ski-resorts/usa/page/4/">4</a> </li> <li> <a href="https://www.skiresort.info/ski-resorts/usa/page/2/">›</a> </li> <li> <a href="https://www.skiresort.info/ski-resorts/usa/page/4/">»</a> </li> </ul>


In [7]:
links_with_text = []
for a in pages.find_all('a', href=True): 
    if a.text: 
        links_with_text.append(a['href'])

In [8]:
links_with_text

['https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/',
 'https://www.skiresort.info/ski-resorts/usa/page/2/',
 'https://www.skiresort.info/ski-resorts/usa/page/3/',
 'https://www.skiresort.info/ski-resorts/usa/page/4/',
 'https://www.skiresort.info/ski-resorts/usa/page/2/',
 'https://www.skiresort.info/ski-resorts/usa/page/4/']

In [9]:
# name = panel.find_all("div", class_= "panel-body middle-padding").find("div", class_="col-sm-11 col-xs-10")

In [10]:
resort_panels = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

In [13]:
# resort_titles = []
# for x in resort_panels:
#     title = x.find("a", class_="h3")
# #     print("-------------------------------------------")
#     resort_titles.append(title)
# #     resort_name.append(resort.text)

In [17]:
resort_names = []
resort_links = []
for x in resort_titles:
    resort_names.append(x.text)
    resort_links.append(x["href"])

In [ ]:
# .find_all("a", class_="h3")

In [27]:
resort_names = []
resort_links = []
for x in resort_panels:
    title = x.find("a", class_="h3")
    resort_names.append(title.text)
    resort_links.append(title["href"])
#     print("-------------------------------------------")
#     resort_titles.append(title)

AttributeError: 'list' object has no attribute 'head'

In [207]:
# GETTING ALL NAMES AND LINKS FOR RESORTS
url = 'https://www.skiresort.info/ski-resorts/usa/'

resort_names = []
resort_links = []

for x in range(1, 5):
    request1 = requests.get(url)

    soup1 = bs(request1.text, 'html.parser')
    
    panel = soup1.find('div', class_= "panel panel-primary")
    resort_panels = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

    for resort in resort_panels:
        title = resort.find("a", class_="h3")
        resort_names.append(title.text)
        resort_links.append(title["href"])
        
    url = f'https://www.skiresort.info/ski-resorts/usa/page/{x+1}'


In [208]:
len(resort_names), len(resort_links)

(525, 525)

In [263]:
test_link = ['https://www.skiresort.info/ski-resort/vail/']
test_links = ['https://www.skiresort.info/ski-resort/vail/', 'https://www.skiresort.info/ski-resort/telluride/',
 'https://www.skiresort.info/ski-resort/beaver-creek/',
 'https://www.skiresort.info/ski-resort/mammoth-mountain/']

In [251]:
# GOING THROUGH RESORT PAGES AND GATHERING RESORT SLOPE LENGTHS
resort_slopes=[]
for x in test_links:

    request1 = requests.get(x)
    soup1 = bs(request1.text, 'html.parser')
    
    stuff = soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart").find("div", class_="description")
    tot_slope= stuff.find("div").text
    slope_type_lengths = stuff.find("div", class_="table-graph-first").find_all("td", class_="distance")
    
    splitter = tot_slope.split()
    
    splits = []
    for y in slope_type_lengths:
        split = float(y.text.split()[0])
        splits.append(split)
    
    slope_lengths={'total': float(splitter[1]), 'easy': splits[0], 'intermediate': splits[1], 'difficult': splits[2]}

    resort_slopes.append(slope_lengths)
resort_slopes

[{'total': 234.0, 'easy': 57.0, 'intermediate': 84.0, 'difficult': 93.0},
 {'total': 88.2, 'easy': 6.7, 'intermediate': 51.7, 'difficult': 29.8},
 {'total': 150.0, 'easy': 28.5, 'intermediate': 64.5, 'difficult': 57.0},
 {'total': 89.8, 'easy': 18.4, 'intermediate': 48.5, 'difficult': 22.9}]

In [258]:
# GOING THROUGH RESORT PAGES AND GATHERING RESORT PRICES
resort_prices = []
for x in resort_links:

    request1 = requests.get(x)

    soup1 = bs(request1.text, 'html.parser')
#     stuff = soup1.find_all("div", class_="detail-links")
#     price = stuff[2].find("td").text

    if (soup1.find(id="selTicketA")):
        
        price = soup1.find(id="selTicketA").text

#     print(price)
#     if price.text: 
#     resort_prices.append(price.text)
        split1 = price.split()
        split2 = float(split1[1].split(',')[0])

        resort_prices.append(split2)
resort_prices

[219.0,
 169.0,
 218.0,
 209.0,
 199.0,
 199.0,
 176.0,
 119.0,
 165.0,
 164.0,
 164.0,
 149.0,
 209.0,
 199.0,
 133.0,
 194.0,
 199.0,
 189.0,
 209.0,
 125.0,
 199.0,
 182.0,
 151.0,
 99.0,
 115.0,
 170.0,
 169.0,
 165.0,
 165.0,
 95.0,
 158.0,
 129.0,
 125.0,
 74.0,
 128.0,
 145.0,
 199.0,
 125.0,
 85.0,
 79.0,
 80.0,
 119.0,
 89.0,
 119.0,
 97.0,
 119.0,
 68.58,
 84.0,
 125.0,
 89.0,
 59.0,
 99.0,
 104.0,
 89.0,
 110.0,
 84.0,
 90.0,
 63.0,
 94.0,
 64.0,
 119.0,
 115.0,
 149.0,
 62.0,
 104.0,
 135.0,
 74.0,
 89.0,
 91.0,
 100.0,
 98.0,
 99.0,
 59.0,
 110.0,
 85.0,
 84.0,
 79.0,
 100.0,
 129.0,
 89.0,
 79.0,
 46.0,
 69.0,
 85.0,
 49.0,
 79.0,
 92.0,
 104.0,
 92.0,
 129.0,
 48.0,
 89.0,
 94.0,
 78.0,
 99.0,
 85.0,
 62.0,
 70.0,
 84.0,
 119.0,
 58.0,
 79.0,
 94.0,
 89.0,
 80.0,
 50.0,
 70.0,
 84.0,
 96.0,
 60.0,
 49.0,
 50.0,
 79.99,
 95.0,
 89.0,
 95.0,
 62.0,
 79.0,
 55.0,
 72.0,
 71.0,
 98.0,
 84.0,
 82.0,
 74.0,
 109.0,
 75.0,
 49.0,
 43.0,
 39.0,
 84.0,
 65.0,
 65.0,
 75.0,
 75.0,

In [259]:
len(resort_prices)

413

In [255]:
resort_prices = []
for x in range(0,50):
    request1 = requests.get(resort_links[x])

    soup1 = bs(request1.text, 'html.parser')
#     stuff = soup1.find_all("div", class_="detail-links")
#     price = stuff[2].find("td")
    price = soup1.find(id="selTicketA")
#     print(price)
#     if price.text: 
    resort_prices.append(price.text)
#     split1 = price.split()
#     split2 = float(split1[1].split(',')[0])

#     resort_prices.append(split2)
resort_prices

AttributeError: 'NoneType' object has no attribute 'text'

In [266]:
# ALL TOGETHER

# NAMES AND LINKS
# url = 'https://www.skiresort.info/ski-resorts/usa/'

# resort_names = []
# resort_links = []

# for x in range(1, 5):
#     request1 = requests.get(url)

#     soup1 = bs(request1.text, 'html.parser')
    
#     panel = soup1.find('div', class_= "panel panel-primary")
#     resort_panels = panel.find_all("div", class_="panel panel-default resort-list-item resort-list-item-image--big")

#     for resort in resort_panels:
#         title = resort.find("a", class_="h3")
#         resort_names.append(title.text)
#         resort_links.append(title["href"])
        
#     url = f'https://www.skiresort.info/ski-resorts/usa/page/{x+1}'

# GOING THROUGH RESORT PAGES AND GATHERING RESORT SLOPE LENGTHS AND PRICES
all_resort_info=[]


for x in range(0, len(test_links)):
# LENGTHS
    resort_info = {'name': resort_names[x], 'link': resort_links[x]}
    
    
    request1 = requests.get(test_links[x])
    soup1 = bs(request1.text, 'html.parser')
    
    stuff = soup1.find("a", class_="detail-links link-img shaded zero-pad-bottom chart").find("div", class_="description")
    tot_slope= stuff.find("div").text
    slope_type_lengths = stuff.find("div", class_="table-graph-first").find_all("td", class_="distance")
    
    splitter = tot_slope.split()
    
    splits = []
    for y in slope_type_lengths:
        split = float(y.text.split()[0])
        splits.append(split)
    
    slope_lengths={'total': float(splitter[1]), 'easy': splits[0], 'intermediate': splits[1], 'difficult': splits[2]}

    resort_info["slopes"] = slope_lengths    
    
# PRICES

    stuff1 = soup1.find_all("div", class_="detail-links")
    price = stuff1[2].find("td").text
    split1 = price.split()
    split2 = float(split1[1].split(',')[0])

    resort_info["price"]=split2

    all_resort_info.append(resort_info)

In [267]:
all_resort_info

[{'name': ' Vail ',
  'link': 'https://www.skiresort.info/ski-resort/vail/',
  'slopes': {'total': 234.0,
   'easy': 57.0,
   'intermediate': 84.0,
   'difficult': 93.0},
  'price': 219.0},
 {'name': ' Telluride ',
  'link': 'https://www.skiresort.info/ski-resort/telluride/',
  'slopes': {'total': 88.2,
   'easy': 6.7,
   'intermediate': 51.7,
   'difficult': 29.8},
  'price': 169.0},
 {'name': ' Beaver Creek ',
  'link': 'https://www.skiresort.info/ski-resort/beaver-creek/',
  'slopes': {'total': 150.0,
   'easy': 28.5,
   'intermediate': 64.5,
   'difficult': 57.0},
  'price': 218.0},
 {'name': ' Mammoth Mountain ',
  'link': 'https://www.skiresort.info/ski-resort/mammoth-mountain/',
  'slopes': {'total': 89.8,
   'easy': 18.4,
   'intermediate': 48.5,
   'difficult': 22.9},
  'price': 209.0}]